In [217]:
from EEG2CodeKeras import (basearchi,
                           basearchitest_batchnorm,
                           basearchi_patchembedding,
                           basearchi_patchembeddingdilation,
                           trueVanilliaEEG2Code,
                           vanilliaEEG2Code,
                           vanilliaEEG2Code2,
                           EEGnet_Inception)
from _utils import make_preds_accumul_aggresive, make_preds_pvalue


from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import make_pipeline
from pyriemann.estimation import XdawnCovariances, Xdawn
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow import keras
import mne
from mne.decoding import Vectorizer
from mne.decoding import CSP
import os

import sys
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt



In [218]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [219]:
fps = 60
sfreq = 500


## Path to the data

In [220]:
sfreq = 500

participant = 'P1'
path = '/'.join(['C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Class4', participant])
# path = '/home/dcas/k.cabrera/Data/SETNOP2'
n_class=4
fps = 60
window_size = 0.25

# file_name = '_'.join([participant, 'mseq40.set'])
# file_name = '_'.join([participant, 'mseq100.set'])
# file_name = '_'.join([participant, 'burst40.set'])
file_name = '_'.join([participant, 'burst100.set'])
# file_name = '/'.join([path,  participant+'_whitemseq.set'])
# file_name = '_'.join([participant, 'burst', 'oi_1.set'])


#### Load channel positions

## Load the raw data and small pre-process
1. Drop the ACC channels and the shitty channels near ears
2. Average re-referencing
4. Extract 2.2s epochs using events

In [221]:
%matplotlib Qt

In [222]:
raw = mne.io.read_raw_eeglab(os.path.join(path, file_name), preload=True, verbose=False)

In [223]:

print(raw.ch_names)
# to_drop = ["P9", "P10", "TP9", "TP10", "10", "21"]
# raw = raw.drop_channels([ch for ch in raw.ch_names if ch in to_drop])
# # raw = raw.drop_channels(["10", "21"])
# keep = ["O1", "O2", "Oz", "P7", "P3", "P4", "P8", "Pz"]
# keep = ["16", "18", "17", "15", "14", "19", "20", "13"] # electrodes to keep
# raw = raw.drop_channels([i for i in raw.ch_names if i not in keep])

# raw = raw.filter(l_freq=50.1, h_freq=49.9, method="iir", verbose=True)
raw = raw.filter(l_freq=1, h_freq=30, method="fir", verbose=True)
# raw = mne.filter.resample(raw,2)
# raw.resample(480, npad='auto')
# Average re-referencing
mne.set_eeg_reference(raw, 'average', copy=False, verbose=False)
#raw = raw.filter(l_freq=5, h_freq=45, method="fir", verbose=True)
n_channels = len(raw.ch_names)
print("Channels :", n_channels)
print(raw.info)

['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz', 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : 32
<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, F9, FC5, FC1, C3, T7, CP5, CP1, Pz, P3, P7, P9, ...
 chs: 32 EEG
 custom_ref_applied: True
 dig: 32 items (32 EEG)
 highpass: 1.0 Hz
 lowpass: 30.0 Hz
 mea

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [224]:
# Strip the annotations that were script to make them easier to process
events, event_id = mne.events_from_annotations(raw, event_id='auto', verbose=False)
to_remove = []
for idx in range(len(raw.annotations.description)):
    if (('collects' in raw.annotations.description[idx]) or
        ('iti' in raw.annotations.description[idx]) or
        (raw.annotations.description[idx] == '[]')):
        to_remove.append(idx)
    else:
        code = raw.annotations.description[idx].split('_')[0]
        lab = raw.annotations.description[idx].split('_')[1]
        code = code.replace('\n', '')
        code = code.replace('[', '')
        code = code.replace(']', '')
        code = code.replace(' ', '')
        raw.annotations.description[idx] = code + '_' + lab

to_remove = np.array(to_remove)
if len(to_remove) > 0:
    raw.annotations.delete(to_remove)
# Get the events
events, event_id = mne.events_from_annotations(raw, event_id='auto', verbose=False)
print(events.shape)
shift = 0.0
# Epoch the data following event
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=shift, \
            tmax=2.2+shift, baseline=(None, None), preload=False, verbose=False)
print(events)

labels = epochs.events[..., -1]
labels -= np.min(labels)
print(epochs.events[..., -1])
data = epochs.get_data()
info_ep = epochs.info
print(epochs)
onset_code = epochs.events[..., 0]

(60, 3)
[[ 65513      0      4]
 [ 67221      0      3]
 [ 68930      0      1]
 [ 70638      0      2]
 [ 72988      0      4]
 [ 74696      0      1]
 [ 76405      0      3]
 [ 78113      0      2]
 [ 80505      0      3]
 [ 82213      0      1]
 [ 83921      0      2]
 [ 85630      0      4]
 [ 88113      0      3]
 [ 89821      0      4]
 [ 91530      0      1]
 [ 93238      0      2]
 [ 95571      0      2]
 [ 97280      0      4]
 [ 98988      0      1]
 [100696      0      3]
 [103338      0      4]
 [105046      0      3]
 [106754      0      2]
 [108463      0      1]
 [110979      0      3]
 [112688      0      1]
 [114396      0      4]
 [116104      0      2]
 [118629      0      2]
 [120338      0      1]
 [122046      0      3]
 [123754      0      4]
 [126438      0      4]
 [128146      0      3]
 [129854      0      2]
 [131563      0      1]
 [134046      0      4]
 [135754      0      3]
 [137463      0      1]
 [139171      0      2]
 [141563      0      2]
 [143271

### Transform a code in `str` to a code in np.array

In [225]:
def code2array(code):
    tmp = []
    for idx, c in enumerate(code[:-2]):
        if c == '5' or c == '.':
            continue
        elif c == '0':
            if code[idx+2] == '5':
                tmp.append(0.5)
            else:
                tmp.append(0)
        else:
            tmp.append(1)
    if code[-1] == '.':
        if code[-2] == '0':
            tmp.append(0)
        else:
            tmp.append(1)
    return np.array(tmp)

### Build a dictionnary that contains all the code in the np.array format

In [226]:
from collections import OrderedDict
codes = OrderedDict()
for k, v in event_id.items():
    code = k.split('_')[0]
    code = code.replace('.','').replace('2','')
    idx = k.split('_')[1]
    if 'randomslowwhite' in file_name:
        codes[v-1] = code2array(code) 
    else:
        codes[v-1] = np.array(list(map(int, code)))

### Define train/test split and windows size
Here we use only the first 7 blocks as calibrition and 8 others would be used as testing

In [227]:
codes

OrderedDict([(0,
              array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
                     1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0,

In [228]:
print(data.shape)

(60, 32, 1101)


In [229]:
sfreq = int(epochs.info['sfreq'])
n_samples_windows = int(window_size*sfreq)
n_trial_per_class = int(len(data)/n_class)

n_cal = 5

data_train = data[:n_class*n_cal]
labels_train = labels[:n_class*n_cal]
data_test = data[n_class*n_cal:]
labels_test = labels[n_class*n_cal:]


### Slice the epoch in windows
The network is not processing full epochs but windows of 250ms. So each epoch is cut into window and the following code (`0` or `1`) is associated as label.

In [230]:
def to_window_old(data, labels):
    length = int((2.2-window_size)*sfreq)
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    print(length)
    print(n_samples_windows)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1 
            y[count] = int(c[code_pos])
            count += 1

    # X = np.expand_dims(X, 1)
    X = X.astype(np.float32)
    y_pred = np.vstack((y,np.abs(1-y))).T
    y = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])
    return X, y

In [231]:
def to_window_new(data, labels, win_size, code_freq, offset=0,
              focus_rising=None, pre_rising=0, post_rising=0):
    length = int((2.2-win_size)*code_freq)
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    Y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    idx_taken = []
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        labels_upsampled = np.repeat(c, code_freq//code_freq)
        labels_upsampled = np.concatenate((np.zeros(int(offset*code_freq), dtype=int), np.array(labels_upsampled)))
        if (focus_rising is not None):
            hi_indices = []
            for idx in range(1, len(labels_upsampled)):
                if (focus_rising is not None) and (labels_upsampled[idx-1] == 0) and (labels_upsampled[idx] == 1):
                    hi_indices.append(idx)
            focused_labels = np.zeros(length)
            pre_rising_frames = int(code_freq*pre_rising)
            post_rising_frames = int(code_freq*post_rising)
            for idx in hi_indices:
                focused_labels[idx:idx+4] = 1
                # focused_labels[idx+1:idx+4] = -1
                # focused_labels[idx-4:idx] = -1
                # idx_to_rm.append([trial_nb*length+idx+post_rising_frames+i for i in range (1,4-post_rising_frames)])
        else:
            focused_labels = labels_upsampled.copy()

        for idx in range(length):
            # print('Xidx:', trial_nb*length+idx, "Tidxm:", idx, 'TidxM:', idx +
            #       n_samples_windows, 'Ltrial', trial[:, idx:idx+n_samples_windows].shape)
            X[trial_nb*length+idx] = trial[:, idx:idx+n_samples_windows]
            Y[trial_nb*length+idx] = focused_labels[idx]
            idx_taken.append(trial_nb*length+idx)
    # X = np.expand_dims(X, 1)
    X = X.astype(np.float32)
    X = np.delete(X,np.where(Y==-1),0)
    idx_taken = np.delete(idx_taken,np.where(Y==-1))
    Y = np.delete(Y,np.where(Y==-1))
    # y_pred = np.vstack((Y,np.abs(1-Y))).T
    # Y = np.delete(Y[:,0],np.where(Y==-1))
    # Y = np.array([1 if (Y >= 0.5) else 0 for Y in y_pred[:,0]])
    return X, Y, idx_taken

In [232]:
window_size = 0.3
n_samples_windows = int(window_size*sfreq)
# X, Y = to_window_old(np.array(data_train[:1]), labels_train[:1])
# X_train, Y_train = to_window_old(data_train, labels_train)
# X_test, Y_test = to_window_old(data_test, labels_test)
# print(Y[:,0])
# X_train, Y_train = to_window_old(data_train, labels_train)#, 0.25, sfreq, 60)
# X_test, Y_test = to_window_old(data_test, labels_test)#, 0.25, sfreq, 60)
X_train, Y_train, frame_train = to_window_new(data_train, labels_train, window_size, 60, focus_rising=True)
X_test, Y_test, frame_test = to_window_new(data_test, labels_test, window_size, 60, focus_rising=True)
# print(codes[labels_test[0]])
print(Y_train.shape)
print(Y_test.shape)
print(frame_train.shape)

(2280,)
(4560,)
(2280,)


In [233]:
def onset_anno(onset_window,label_window,onset_code,nb_seq_min,nb_seq_max,code_freq,sfreq,win_size):
    assert(sfreq!=0)
    new_onset = []
    new_onset_0 = []
    current_code = 0
    onset_code = np.ceil(onset_code*code_freq/sfreq)
    nb_seq_min-=1
    onset_shift = onset_code[current_code+nb_seq_min]
    time_trial = (2.2-win_size)
    # onset_window = np.arange(0,time_trial*code_freq*(nb_seq_max-nb_seq_min)-1,1,dtype=int)
    for i,o in enumerate(onset_window):
        if label_window[i]==1:
            # print(i)
            if current_code==nb_seq_max-1-nb_seq_min:
                new_onset.append(o+onset_shift)
            else:
                if o+onset_shift >= onset_code[current_code+nb_seq_min]+time_trial*code_freq:
                    current_code+=1
                    onset_shift = onset_code[current_code+nb_seq_min]-time_trial*code_freq*current_code
                new_onset.append(o+onset_shift)
        else:
            if current_code==nb_seq_max-1-nb_seq_min:
                new_onset_0.append(o+onset_shift)
            else:
                if o+onset_shift >= onset_code[current_code+nb_seq_min]+time_trial*code_freq:
                    current_code+=1
                    onset_shift = onset_code[current_code+nb_seq_min]-time_trial*code_freq*current_code
                new_onset_0.append(o+onset_shift)
    
    # modified_onset_code = [onset_code[i]-time_trial*sfreq*i for i in range(nb_seq_min,nb_seq_max)]
    # new_onset_0 = np.concatenate([np.arange(onset_code[i],onset_code[i]+time_trial*sfreq,sfreq//60) for i in range(nb_seq_min,nb_seq_max)])
    new_onset_0 = np.array(list(filter(lambda i: i not in new_onset, new_onset_0)))
    # print(new_onset_0.shape)
    return np.array(new_onset)/code_freq, np.array(new_onset_0)/code_freq
            

In [234]:
# print(Y_train)
# print(np.where(Y_train==1)[0]*sfreq//60)
# print(frame_train)
onset_train,onset_train_0 = onset_anno(frame_train,Y_train,onset_code,1,n_class*n_cal,60,500,window_size)
print(onset_train,onset_train_0)
anno = mne.Annotations(onset_train,0.001*sfreq//60,"1")
anno.append(onset_train_0,0.001*sfreq//60,"0")
onset_test,onset_test_0 = onset_anno(frame_test,Y_test,onset_code,n_class*n_cal+1,60,60,500,window_size)
anno.append(onset_test,0.001*sfreq//60,"1")
anno.append(onset_test_0,0.001*sfreq//60,"0")
raw = raw.set_annotations(anno)
# raw.plot(n_channels=15, duration=300)

[131.05       131.06666667 131.08333333 131.1        131.53333333
 131.55       131.56666667 131.58333333 132.         132.01666667
 132.03333333 132.05       132.41666667 132.43333333 132.45
 132.46666667 132.85       132.86666667 132.88333333 132.9
 134.7        134.71666667 134.73333333 134.75       135.13333333
 135.15       135.16666667 135.18333333 135.6        135.61666667
 135.63333333 135.65       135.9        135.91666667 135.93333333
 135.95       136.33333333 138.25       138.26666667 138.28333333
 138.3        138.63333333 138.65       138.66666667 138.68333333
 138.93333333 138.95       138.96666667 138.98333333 139.21666667
 139.23333333 139.25       139.26666667 139.55       139.56666667
 139.58333333 139.6        141.58333333 141.6        141.61666667
 141.63333333 141.98333333 142.         142.01666667 142.03333333
 142.26666667 142.28333333 142.3        142.31666667 142.63333333
 142.65       142.66666667 142.68333333 143.1        143.11666667
 143.13333333 143.15   

In [235]:
events, event_id = mne.events_from_annotations(raw)
print(event_id,events)
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=0, \
            tmax=0.6, baseline=(None, None), preload=False, verbose=False)
# epochs.plot()

Used Annotations descriptions: ['0', '1']
{'0': 1, '1': 2} 

[[ 65517      0      1]
 [ 65525      0      2]
 [ 65533      0      2]
 [ 65542      0      2]
 [ 65550      0      2]
 [ 65558      0      1]
 [ 65567      0      1]
 [ 65575      0      1]
 [ 65583      0      1]
 [ 65592      0      1]
 [ 65600      0      1]
 [ 65608      0      1]
 [ 65617      0      1]
 [ 65625      0      1]
 [ 65633      0      1]
 [ 65642      0      1]
 [ 65650      0      1]
 [ 65658      0      1]
 [ 65667      0      1]
 [ 65675      0      1]
 [ 65683      0      1]
 [ 65692      0      1]
 [ 65700      0      1]
 [ 65708      0      1]
 [ 65717      0      1]
 [ 65725      0      1]
 [ 65733      0      1]
 [ 65742      0      1]
 [ 65750      0      1]
 [ 65758      0      1]
 [ 65767      0      2]
 [ 65775      0      2]
 [ 65783      0      2]
 [ 65792      0      2]
 [ 65800      0      1]
 [ 65808      0      1]
 [ 65817      0      1]
 [ 65825      0      1]
 [ 65833      0      1]
 [ 65842      0      1]
 [ 65850      0      1]
 [ 65858      0 

In [236]:
epochs_train1 = mne.Epochs(raw,events, 2,tmin=-0.01,tmax=window_size)
epochs_train0 = mne.Epochs(raw,events, 1,tmin=-0.01,tmax=window_size)
# epochs_train1.plot()

Not setting metadata
1155 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Not setting metadata
5685 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated


In [237]:
evoked_1 = list(epochs_train1.iter_evoked())
# evoked_1.plot(picks="Oz")
evoked_0 = list(epochs_train0.iter_evoked())
# evoked_0.plot(picks="Oz")
evokeds = dict(one=evoked_1,zero=evoked_0)
mne.viz.plot_compare_evokeds(evokeds, combine="mean", picks="Oz")

combining channels using "mean"


C:\Users\s.velut\AppData\Local\Temp\ipykernel_18012\2255816275.py:6: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(evokeds, combine="mean", picks="Oz")


combining channels using "mean"


C:\Users\s.velut\AppData\Local\Temp\ipykernel_18012\2255816275.py:6: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  mne.viz.plot_compare_evokeds(evokeds, combine="mean", picks="Oz")


[<Figure size 1000x750 with 1 Axes>]

In [238]:
epochs = mne.Epochs(raw,events,event_id,tmin=-0.01,tmax=0.3)
X=epochs.get_data()
print(frame_train.shape)
label = epochs.events[...,-1]
label.shape
size_train = frame_train.shape[0] #int((2.2-window_size)*n_class*n_cal*60)
X_train = X[:size_train]
X_test = X[size_train:]
Y_train = label[:size_train]-1
Y_test = label[size_train:]-1

Not setting metadata
6840 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 6840 events and 156 original time points ...
0 bad epochs dropped
(2280,)


### Normalization using stats from the train set

In [239]:
X_std = X_train.std(axis=0)
X_train /= X_std + 1e-8
X_std = X_test.std(axis=0)
X_test /= X_std + 1e-8

### Balance classes
Our classes are unbalanced, there are more `1` than `0` in the train set (the stimulation is more often ON than OFF).  
We will use a random under sampler to make it balance.

In [240]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(len(Y_train[Y_train[:] == 1]))
print(len(Y_train[Y_train[:] == 2]))
print(Y_test.shape)
print(len(Y_test[Y_test[:] == 2]))


(2280, 32, 156)
(4560, 32, 156)
(2280,)
385
0
(4560,)
0


In [241]:
print(len(Y_train[Y_train[:] == 1]))
print(len(Y_train[Y_train[:] == 2]))
print(len(Y_test[Y_test[:] == 1]))
print(len(Y_test[Y_test[:] == 2]))

385
0
770
0


### Pick an architecture

In [242]:
#  clf = basearchi_patchembedding(windows_size = n_samples_windows, n_channel_input = n_channels) # other stim
# clf = basearchi_patchembeddingdilation(windows_size = n_samples_windows, n_channel_input = n_channels) # burst
# clf = vanilliaEEG2Code(windows_size = n_samples_windows, n_channel_input = n_channels) # burst
# clf = trueVanilliaEEG2Code(windows_size = n_samples_windows, n_channel_input = n_channels) # burst
# clf = basearchitest(windows_size = n_samples_windows, n_channel_input = n_channels)
#c lf = basearchitest_batchnorm(windows_size = n_samples_windows, n_channel_input = n_channels)
# clf = EEGnet_Inception(windows_size = n_samples_windows, n_channel_input = n_channels)
# clf = basearchi(windows_size = n_samples_windows, n_channel_input = n_channels)

from sklearn.svm import SVR


clf = make_pipeline(XdawnCovariances(nfilter=6, estimator="oas", xdawn_estimator="lwf"),MDM())
clf2 = make_pipeline(XdawnCovariances(nfilter=6, estimator="oas", xdawn_estimator="lwf"),
    TangentSpace(),
    LDA(solver="lsqr", shrinkage="auto"))
clf3 = make_pipeline(XdawnCovariances(nfilter=6, estimator="oas", xdawn_estimator="lwf"),
    TangentSpace(),
    svm.SVC())
clf4 = make_pipeline(Xdawn(nfilter=4, estimator="lwf"),Vectorizer(),LDA(solver="lsqr", shrinkage="auto"))
clf5 = make_pipeline(CSP(n_components=4, reg=None, log=True, norm_trace=False),svm.SVC())

# xdawn = XdawnCovariances(nfilter=3, estimator="lwf", xdawn_estimator="scm")
# clf = MDM()


# clf.steps

In [243]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)



(2280, 32, 156)
(2280,)
(4560, 32, 156)
(4560,)


### Cut the train in train and valid
Also set some HP of the network

In [244]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, shuffle=True)
batchsize = 128 #128 # 64 for burst
epochs = 45 #45 # 20 for burst

### Attach an optimizer and train the network

res_X_train=[]
for i in range(len(X_train)):
    res_X_train.append(np.array(X_train[i:i+1]).reshape(n_samples_windows, n_channels, 1))
res_X_val=[]
for i in range(len(x_val)):
    res_X_val.append(np.array(x_val[i:i+1]).reshape(n_samples_windows, n_channels, 1))

np.array(res_X_val).shape

np.array(res_X_train).shape

In [245]:
# lr = 1e-3
# weight_decay = 1e-4
# optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
# clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
# X_trainp = xdawn.fit_transform(X_train,y_train)
# X_valp = xdawn.fit_transform(x_val,y_val)
# X_testp = xdawn.fit_transform(X_test,Y_test)>
history = clf.fit(np.array(x_train), y_train)
history2 = clf2.fit(np.array(x_train), y_train)
history3 = clf3.fit(np.array(x_train), y_train)
history4 = clf4.fit(np.array(x_train), y_train)

reg_x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
# history_r1 = rm.fit(np.array(reg_x_train), y_train)
# history5 = clf5.fit(np.array(x_train), y_train)
keras.backend.clear_session()


In [246]:
from sklearn.metrics import balanced_accuracy_score


print("RG")
print(balanced_accuracy_score(y_train,history.predict(x_train)))
print(balanced_accuracy_score(y_val,history.predict(x_val)))
print(balanced_accuracy_score(Y_test,history.predict(X_test)))

print("RG+LDA")
print(balanced_accuracy_score(y_train,history2.predict(x_train)))
print(balanced_accuracy_score(y_val,history2.predict(x_val)))
print(balanced_accuracy_score(Y_test,history2.predict(X_test)))

print("RG+SVC")
print(balanced_accuracy_score(y_train,history3.predict(x_train)))
print(balanced_accuracy_score(y_val,history3.predict(x_val)))
print(balanced_accuracy_score(Y_test,history3.predict(X_test)))

print("Xdw+LDA")
print(balanced_accuracy_score(y_train,history4.predict(x_train)))
print(balanced_accuracy_score(y_val,history4.predict(x_val)))
print(balanced_accuracy_score(Y_test,history4.predict(X_test)))

# print("CSP+LDA")
# print(history5.score(x_train,y_train))
# print(history5.score(x_val,y_val))
# print(history5.score(X_test,Y_test))
# print(history.predict(X_trainp)==y_train)

RG
0.8688004873157651
0.8296694966646452
0.8016225199602509
RG+LDA
0.8754047881704516
0.7635688295936931
0.7550765856834458
RG+SVC
0.9690361907313714
0.7985142510612493
0.6943768632422986
Xdw+LDA
0.8887735258593441
0.8716646452395391
0.8233098036528115


### Model and accuracy and loss
Just check that the model learnt something

In [247]:
plt.figure(figsize=(12,12))
# summarize history for accuracy
plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# summarize history for loss
plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
print()


AttributeError: 'Pipeline' object has no attribute 'history'

filename = ''
np.save(filename + "std_from_calibration", X_std)
model_filename = os.path.join(os.getcwd(), filename + '0.7' + "trainedmodel")
# Save the model if calibration was done
clf.save(model_filename)
keras.backend.clear_session()

## Vizualize learned filters
### Raw vizualiation of 1D convolutinal kernel of the first layer --> spatial filters

In [ ]:
# # We first visualize the learned patch embeddings.
# patch_embeddings = clf.layers[0].get_weights()[0]
# weights = patch_embeddings
# # First, apply min-max normalization to the
# # given weights to avoid isotrophic scaling.
# p_min, p_max = weights.min(), weights.max()
# weights = (weights - p_min) / (p_max - p_min)

# # Visualize all the filters.
# num_filters = 10
# plt.figure(figsize=(16, 2))
# idx = 1
# for i in range(num_filters):
#     current_weight = weights[:, :, :, i]
#     if current_weight.shape[-1] == 1:
#         current_weight = current_weight.squeeze()
#     ax = plt.subplot(1, 10, idx)
#     ax.set_xticks([])
#     ax.set_yticks([])
#     plt.plot(current_weight)
#     idx += 1

### Viz of the corresponding topo maps

In [ ]:
# spatialfilts = clf.get_layer("conv2d").get_weights()[0]
# spatialfilts = np.squeeze(spatialfilts)
# spatialfilts = np.swapaxes(spatialfilts, 1, 0)

### Predict on the test set
The predictions are made on windows to regress the code.  
Here we divide the prediction in 10 fold to avoid OOM from the GPU.

In [ ]:
keras.backend.clear_session()

In [ ]:
keras.backend.clear_session()
y_pred = history4.predict(X_test)
y_pred = np.array(y_pred)

In [ ]:
# y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
# y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])
y_pred_norm = y_pred
y_test_norm = Y_test

In [ ]:
import sklearn.metrics
import math
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test_norm, y_pred_norm).ravel()
print("True positives:", tp)
print("True negatives:", tn)
print("False positives:", fp)
print("False negatives:", fn)
print("Accuracy:", (tp+tn)/len(y_test_norm))
print("Sensitivity:", sen:=tp/(tp+fn))
print("Precision:", pre:=tp/(tp+fp))
print("Fowlkes-Mallows:", math.sqrt(sen*pre))


True positives: 407
True negatives: 3545
False positives: 335
False negatives: 393
Accuracy: 0.8444444444444444
Sensitivity: 0.50875
Precision: 0.5485175202156334
Fowlkes-Mallows: 0.5282596789550604


### Convert prediction on windows (regressed code) to label prediction

It is offline and synchronous, so we will:
1. First create a `code_buffer` that contains the regressed code on the full epoch (2.2s - the last window)
    - The refresh rate of the EEG device (500 Hz) is faster than the refress rate of the screen (60Hz), so we average predictions (500/60~8 samples) so they correspond to each flip of the screen.
2. Starting from `min_len` (in number of samples), we compute Pearson correalation with the bank of templates code to find the closest one
    - If the most correlated code has a significantely bigger correlation compared to the second one (50% bigger) and the p_value is significative then the trial is classified and we move to the next one
    - If the thresholds are not reached, then we add samples (with a step of 3) to have a longer trial and re-do the computation
    - The thresholds can never been reached in 2.2s, then the trial is not classified.

In [ ]:
# p_best_score = 0
# p_best_ratio = 0
# p_best_acc = 0
# p_best_values = ()

# for p in np.logspace(0,-5, 6):
#     for dr in np.linspace(0.2, 0.8, 7):
#         labels_pred, _, mean_long = make_preds_pvalue(y_pred, codes, min_len=70, sfreq=sfreq, obj_p=p)
#         ratio = len(labels_pred[labels_pred != -1])/len(labels_pred)
#         accuracy = accuracy_score(labels_test[labels_pred != -1], labels_pred[labels_pred != -1])
#         score = ratio*accuracy
#         if ratio >= p_best_ratio:
#             p_best_score = score
#             p_best_acc = accuracy
#             p_best_ratio = ratio
#             p_best_values = (p, dr)
#         print("==========", p, "+", dr,':',ratio,'--',accuracy,'--',score, '--', np.mean(mean_long))
# print("Best score", p_best_score)
# print("Best values", p_best_values)            
        

### Compute accuracy score and accuracy score when a prediction is made (discard not classified trials)

### Other classification method
Same as before but the classification method is different. Instead of thresholds to reach, if when increasing trial lengt a code correllated the most 40 times in a row then the trial is labeled.

In [ ]:
# import importlib
# importlib.reload(_utils)
# from _utils import make_preds_accumul_aggresive, make_preds_pvalue



In [ ]:
acc_best_score=0
acc_best_ratio=0
acc_best_acc=0
acc_best_value = 0
# for minlen in np.linspace(10, 60, 10):
for cons in [15, 20, 25, 30, 35, 40, 45, 50, 55, 60]:
    
    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=60, consecutive=cons, window_size=window_size
    )
    print(labels_pred_accumul)
    print(labels_test)
    ratio = np.round(len(labels_pred_accumul[labels_pred_accumul != -1])/len(labels_pred_accumul), 2)
    accuracy = np.round(accuracy_score(labels_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    score = np.round(ratio*accuracy,2)
    print("==========", cons, ':', ratio, '--', accuracy,
        '--', score, '--', np.mean(mean_long_accumul))
    if accuracy > acc_best_acc:
        acc_best_score = score
        acc_best_value = cons
        acc_best_acc = accuracy
        acc_best_ratio = ratio

print("Best accuracy", acc_best_acc)
print("Best ratio", acc_best_ratio)
print("Best score", acc_best_score)
print("Best values", acc_best_value)


[0 2 1 3 3 1 2 2 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 1 2 1 3 2 1 3 2 0 2 2 2 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 15 : 1.0 -- 0.68 -- 0.68 -- 0.8083333333333333


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 2 2 1 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 0 2 1 3 2 1 3 2 0 0 2 3 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 20 : 1.0 -- 0.8 -- 0.8 -- 0.9483333333333335
[0 2 1 3 3 2 2 1 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 0 2 1 3 2 1 3 2 0 0 2 3 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 25 : 1.0 -- 0.8 -- 0.8 -- 1.0316666666666667


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 1 2 1 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 0 2 1 3 2 1 3 2 0 0 2 3 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 30 : 1.0 -- 0.78 -- 0.78 -- 1.105128205128205


...

[0 2 1 3 3 1 2 1 1 0 1 3 3 2 2 0 0 3 1 2 3 2 1 0 2 1 3 2 1 3 2 0 0 1 3 2 0
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 50 : 1.0 -- 0.85 -- 0.85 -- 1.4171568627450981


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 1 2 1 1 0 1 3 3 1 2 0 0 3 1 2 3 2 1 0 2 1 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 55 : 1.0 -- 0.9 -- 0.9 -- 1.5086021505376344


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 1 2 1 1 0 1 3 3 1 2 0 0 3 1 2 3 2 1 0 2 1 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 60 : 1.0 -- 0.9 -- 0.9 -- 1.5811111111111107
Best accuracy 0.9
Best ratio 1.0
Best score 0.9
Best values 55


In [ ]:
accuracy_score(labels_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1])

0.85

In [ ]:
labels_pred_accumul, aaaaa, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=sfreq, consecutive=15, window_size=window_size
    )

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
len(labels_pred_accumul)


In [ ]:
codes